In [2]:
import matplotlib
import matplotlib.pylab as plt
import matplotlib.colors as colors
from glob import glob
import os
import pathlib
%run depth_profile_v1.py
%run calibration_Cube_v4.py
%matplotlib

Using matplotlib backend: Qt5Agg


In [45]:
# open measurement results
path_ = '20200622_rhizosphere/output_fullcalibration20200806/TPTBP/light_adaption_run2/'
path_res = '20200622_rhizosphere/output_fullcalibration20200806/results/TPTBP/'
save_op = dict({'type': ['png', 'svg', 'tiff'], 'dpi': 300})

In [4]:
# parameter
cmap = 'inferno'
lw = 2.5
cutoff = 5. # percent
name_singles = ['Pt-TPTBP+MY', 'Pd-TPTBP+MY'] # 01072020

cm = input('Scale for conversion of px into cm of a known disance: (px, cm) ') # (825, 7.5)

Scale for conversion of px into cm of a known disance: (px, cm) 825, 7.5


In [131]:
# %run depth_profile_v1.py
def depth_profile(file, cm_conv, name_single=None, max_calib=True, cutoff=5., lw=2.5, cmap='inferno',
                 offset=(0, 0)):
    [dict_header, singleID, dic_raw, dict_calib, iratio1, iratio2, analyte1, analyte2, 
     dic_figures] = calib.load_evaluation(file=file)
    dic_calib = dict({'header': dict_header, 'singleID': singleID, 'raw': dic_raw, 
                      'dict_calib': dict_calib, 'Intensity 1': iratio1, 'Intensity 2': iratio2,
                      'Analyte 1': analyte1, 'Analyte 2': analyte2})
    
    if name_single is None:
        ls_ind = singleID
    else:
        ls_ind = name_single
    [d_cm, sens1, sens2, analyte1_px, 
     analyte2_px] = convert_px_cm(cm=cm_conv, analyte1=analyte1, analyte2=analyte2)

    # =================================================================
    # plot image frame for depth profile selection
    print('Draw line for depth profile in second plot ', singleID[1])
    if offset:
        print(21)
        sens1_ = sens1 - offset[0]
        sens2_ = sens2 - offset[1]
    [toggle_selector, fig_op, 
     fig_op2] = plot.plot_measurement_results_depth_analysis(sens1=sens1_, sens2=sens2_, 
                                                             dict_calib=dict_calib, cmap=cmap,
                                                             singleID=ls_ind, cutoff=cutoff,
                                                             max_calib=max_calib)

    return dic_calib, d_cm, dict_calib, toggle_selector, sens1_, sens2_, analyte1_px, analyte2_px, fig_op, fig_op2

for i in glob(path_ + '/*.hdf5'):
    print('Analyse ', i.split('\\')[-1])
    [dic_calib, d_cm, dict_calib, toggle_selector, sens1, sens2, analyte1, analyte2, fig_op, 
     fig_op2] = depth_profile(file=i, cm_conv=cm, cutoff=cutoff, lw=lw, cmap=cmap,
                              name_single=name_singles, max_calib=500, 
                              offset=(28, 22))

Analyse  2020810_20200622_optodes_100ms_run2_cube_radiometric_simplifiedSV_ratio_Analysis_run0.hdf5
Draw line for depth profile in second plot  Pt-TPTBP+MY
21
startposition: (109, 132)
endposition  : (110, 275)
used button  :  MouseButton.LEFT


In [133]:
%run depth_profile_v1.py

def print_profile(toggle_selector, analyte1, analyte2, singleID, d_cm, lw, name_id, plt_limit):
    [i1, i2, depth1, depth2, drawLine, slope, 
     y_absz] = valuesFromLine(toggle_selector=toggle_selector, d_px=d_cm, analyte1=analyte1, 
                              analyte2=analyte2, lw_profile=lw)
    
    # plotting results - depth profile
    fig, ax = plot.plot_depthProfile(singleID=name_id, depth1=depth1, depth2=depth2, 
                                     plt_limit=plt_limit)
    
    return depth1, depth2, fig, ax, drawLine, slope, y_absz, i1, i2

lw_profile = 10.
[depth1, depth2, fig, ax, drawLine, slope, y_absz, i1, 
 i2] = print_profile(toggle_selector=toggle_selector, analyte1=analyte1-25, analyte2=analyte2-25,
                     singleID=dic_calib['singleID'], d_cm=d_cm, lw=lw_profile, name_id=name_singles, 
                     plt_limit=[-25, 400])

{'end_px': (109, 132), 'start_px': (110, 275)} slope None abscissa [132. 133. 134. 135. 136. 137. 138. 139. 140. 141. 142. 143. 144. 145.
 146. 147. 148. 149. 150. 151. 152. 153. 154. 155. 156. 157. 158. 159.
 160. 161. 162. 163. 164. 165. 166. 167. 168. 169. 170. 171. 172. 173.
 174. 175. 176. 177. 178. 179. 180. 181. 182. 183. 184. 185. 186. 187.
 188. 189. 190. 191. 192. 193. 194. 195. 196. 197. 198. 199. 200. 201.
 202. 203. 204. 205. 206. 207. 208. 209. 210. 211. 212. 213. 214. 215.
 216. 217. 218. 219. 220. 221. 222. 223. 224. 225. 226. 227. 228. 229.
 230. 231. 232. 233. 234. 235. 236. 237. 238. 239. 240. 241. 242. 243.
 244. 245. 246. 247. 248. 249. 250. 251. 252. 253. 254. 255. 256. 257.
 258. 259. 260. 261. 262. 263. 264. 265. 266. 267. 268. 269. 270. 271.
 272. 273. 274. 275.]
vertical
vertical


In [134]:
# saving results
if os.path.isdir(path_res) == False:
    pathlib.Path(path_res).mkdir(parents=True, exist_ok=True)
        
for t in save_op['type']:
    fig.savefig(path_res + 'SingleIndicator_DepthProfile_run2_RoI2.' + t, dpi=save_op['dpi'],
                transparent=True)

### combine information for dual sensing

In [59]:
d = _load_calibration_dual(path_calib=dic_calib['header']['calibration file'], simply=True,
                           ratiometric=bool(dic_calib['header']['ratiometric evaluation']))
df_calFit = pd.concat(d['Fit']['best Fit']['Dual+MY'], axis=1)
dd_int = d['data']['integral']['Dual+MY']
dd_ratio = dd_int / dd_int.loc[0]

dt = df_calFit.min()/df_calFit
limit = dt[round(dt['Pt-TPTBP+MY'] / dt['Pd-TPTBP+MY'], 2)[0] <=1.00].index[0]

limit = 70.

In [60]:
# plotting
ls_colors = ['#ff8c00', '#347474']
fig, ax = plt.subplots(figsize=(5,5), nrows=2, sharex=True)
axR = ax[0].twinx()
axT = ax[0].twiny()
axR.get_shared_y_axes().join(axR, ax[0])
axT.get_shared_x_axes().join(axT, ax[0])
axR1 = ax[1].twinx()
axT1 = ax[1].twiny()
axR1.get_shared_y_axes().join(axR1, ax[1])
axT1.get_shared_x_axes().join(axT1, ax[1])

for en, c in enumerate(df_calFit.columns):
    ax[0].plot(df_calFit[c].max()/df_calFit[c], lw=1., ls='-.', color=ls_colors[en])
    ax[1].plot(df_calFit[c].min()/df_calFit[c], lw=1., ls='-.', color=ls_colors[en])

ax[0].legend(df_calFit.columns.levels[0], fontsize=8, frameon=True, fancybox=True)
for en, c in enumerate(df_calFit.columns):
    ax[0].plot(1/dd_ratio[c[0]], lw=0., marker='o', fillstyle='none', ms=4, color=ls_colors[en])
    ax[1].plot(dd_ratio[c[0]].min()/dd_ratio[c[0]], lw=0., marker='o', fillstyle='none', ms=4, 
               color=ls_colors[en])
ax[1].axvline(limit, lw=1., color='crimson')

ax[0].tick_params(axis='both', which='both', direction='out')
ax[1].tick_params(axis='both', which='both', direction='out')
axR.tick_params(axis='both', which='both', direction='in', labelcolor='white')
axT.tick_params(axis='both', which='both', direction='in', labelcolor='white')
axR1.tick_params(axis='both', which='both', direction='in', labelcolor='white')
axT1.tick_params(axis='both', which='both', direction='in', labelcolor='white')
ax[1].set_xlabel('$O_2$ concentration [hPa]')
ax[0].set_ylabel('Integral $R_0/R$')
ax[1].set_ylabel('Rel. integral')
plt.tight_layout()

##### combine indicator results accordingly

In [135]:
analyte_comb = analyte1.copy()

if name_singles[0].startswith('Pt'):
    print('delete all values < ', limit)
    for col in analyte_comb.columns:
        analyte_comb.loc[~(analyte_comb[col] >= limit), col] = 0.
else:
    print('delete all values > ', limit) 
    for col in analyte_comb.columns:
        analyte_comb.loc[~(analyte_comb[col] <= limit), col] = 0.
# ----------------------------
print('\nre-fill nan values with analyte 2')   

for col in analyte2.columns:
        analyte2.loc[~(analyte2[col] <= limit), col] = 0.
analyte_joint = analyte2 + analyte_comb

delete all values <  70.0

re-fill nan values with analyte 2


In [136]:
# balancing outcome (smoothing, cick out values out of range,...)
for col in analyte_joint.columns:
    analyte_joint.loc[~(analyte_joint[col] > 0), col] = np.nan
    
if analyte_joint.isnull().any().any() == True:
    # find position of nan values
    dic_index = dict(map(lambda c: (c, analyte_joint[analyte_joint[c].isnull()].index.tolist()), 
                         analyte_joint.columns))
    df_joint = mean_values_around_nan(dic_index, analyte_joint)  
else:
    df_joint = analyte_joint

In [137]:
[i_sup, i2, depth_sup, df2, drawLine, slope, 
 y_absz] = valuesFromLine(drawLine=drawLine, d_px=d_cm, analyte1=df_joint, lw_profile=lw_profile)

{'end_px': (109, 132), 'start_px': (110, 275)} slope None abscissa [132. 133. 134. 135. 136. 137. 138. 139. 140. 141. 142. 143. 144. 145.
 146. 147. 148. 149. 150. 151. 152. 153. 154. 155. 156. 157. 158. 159.
 160. 161. 162. 163. 164. 165. 166. 167. 168. 169. 170. 171. 172. 173.
 174. 175. 176. 177. 178. 179. 180. 181. 182. 183. 184. 185. 186. 187.
 188. 189. 190. 191. 192. 193. 194. 195. 196. 197. 198. 199. 200. 201.
 202. 203. 204. 205. 206. 207. 208. 209. 210. 211. 212. 213. 214. 215.
 216. 217. 218. 219. 220. 221. 222. 223. 224. 225. 226. 227. 228. 229.
 230. 231. 232. 233. 234. 235. 236. 237. 238. 239. 240. 241. 242. 243.
 244. 245. 246. 247. 248. 249. 250. 251. 252. 253. 254. 255. 256. 257.
 258. 259. 260. 261. 262. 263. 264. 265. 266. 267. 268. 269. 270. 271.
 272. 273. 274. 275.]
vertical


In [138]:
fig_depth, ax_depth = plt.subplots()
axR = ax_depth.twinx()
axT = ax_depth.twiny()
axR.get_shared_y_axes().join(axR, ax_depth)
axT.get_shared_x_axes().join(axT, ax_depth)

if slope:
    if slope < 80:
        ax_depth.plot(depth_sup, color='#46505A')
    else:
        ax_depth.plot(depth_sup, color='#46505A')
else:
    ax_depth.plot(depth_sup)
    pd.DataFrame(depth1[0].to_numpy(),
                 index=df_joint.loc[i1.index, i1.columns].mean(axis=1))

#ax_depth.set_xlim(-25, 600)
ax_depth.tick_params(axis='both', which='both', direction='out')
axR.tick_params(axis='both', which='both', direction='in', labelcolor='white')
axT.tick_params(axis='both', which='both', direction='in', labelcolor='white')

ax_depth.set_xlabel('$O_2$ concentration [hPa]')
ax_depth.set_ylabel('depth [cm]')

plt.tight_layout()

In [140]:
# plot joint information
W, H = np.meshgrid(sens1.columns, sens1.index)
o2_calc = np.array(df_joint)
m = np.ma.masked_where(np.isnan(o2_calc), o2_calc)

fig_op, ax_op = plt.subplots(figsize=(7, 5))
pcm = ax_op.pcolormesh(W, H, m, cmap=cmap, norm=colors.Normalize(vmin=0, vmax=500))
fig_op.colorbar(pcm, ax=ax_op, extend='max')

Xflat, Yflat, Zflat = W.flatten(), H.flatten(), m.flatten()
def fmt(x, y):
    # get closest point with known data
    dist = np.linalg.norm(np.vstack([Xflat - x, Yflat - y]), axis=0)
    idx = np.argmin(dist)
    z = Zflat[idx]
    return 'x={x:.5f}  y={y:.5f}  z={z:.5f}'.format(x=x, y=y, z=z)
ax_op.format_coord = fmt

plt.tight_layout()

In [141]:
for t in save_op['type']:
    fig_op.savefig(path_res + 'DualIndicator_ImageFrame_RoI.' + t, dpi=save_op['dpi'], transparent=True)
    #fig_depth.savefig(path_res + 'DualIndicator_DepthProfile_RoI2.' + t, dpi=save_op['dpi'], 
    #                  transparent=True)